In [3]:
import torch
from datasets import load_dataset
import warnings
warnings.filterwarnings("ignore")

In [16]:
# Load training dataset, only the first 50,000 examples
train_dataset = load_dataset("wmt16", "de-en", split="train[:50000]")

# Load validation and test datasets
validation_dataset = load_dataset("wmt16", "de-en", split="validation")
test_dataset = load_dataset("wmt16", "de-en", split="test")

# Setup-2A


In [17]:
from typing import Iterable
import torch
import math
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import Multi30k
from typing import Iterable, List


# Define function to yield tokens from the dataset
def yield_tokens(data_iter: Iterable, language: str):
    for data_sample in data_iter:
        yield token_transform[language](data_sample['translation'][language])

# Set up tokenizers
token_transform = {}
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')


# Build vocabulary
vocab_transform = {}
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
vocab_transform[SRC_LANGUAGE] = build_vocab_from_iterator(yield_tokens(train_dataset, 'de'), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
vocab_transform[TGT_LANGUAGE] = build_vocab_from_iterator(yield_tokens(train_dataset, 'en'), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
vocab_transform[SRC_LANGUAGE].set_default_index(UNK_IDX)
vocab_transform[TGT_LANGUAGE].set_default_index(UNK_IDX)

In [18]:
import torch
import math
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import Multi30k
from typing import Iterable, List

# Define the PositionalEncoding module
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout: float, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])


class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)


# Define the Transformer model
class TransformerModel(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, emb_dim=234, nhead=6, num_encoder_layers=5, num_decoder_layers=5, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.embedding_dim = emb_dim
        self.transformer = nn.Transformer(
            d_model=emb_dim, nhead=nhead, num_encoder_layers=num_encoder_layers, 
            num_decoder_layers=num_decoder_layers, dropout=dropout, batch_first=True)
        self.src_token_emb = nn.Embedding(src_vocab_size, emb_dim)
        self.tgt_token_emb = nn.Embedding(tgt_vocab_size, emb_dim)
        self.linear = nn.Linear(emb_dim, tgt_vocab_size)
        self.positional_encoding = PositionalEncoding(emb_dim, dropout=dropout)


    def forward(self, src, tgt, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_padding_mask):
        src_emb = self.positional_encoding(self.src_token_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_token_emb(tgt))
        output = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None, src_padding_mask, tgt_padding_mask, memory_padding_mask)
        output = self.linear(output)
        return output
    
    def encode(self, src, src_mask):
        return self.transformer.encoder(self.positional_encoding(self.src_token_emb(src)), src_mask)

    def decode(self, tgt, memory, tgt_mask):
        return self.transformer.decoder(self.positional_encoding(self.tgt_token_emb(tgt)), memory, tgt_mask)


In [19]:
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'
vocab_transform = {}
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
vocab_transform[SRC_LANGUAGE] = build_vocab_from_iterator(yield_tokens(train_dataset, 'de'), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
vocab_transform[TGT_LANGUAGE] = build_vocab_from_iterator(yield_tokens(train_dataset, 'en'), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
vocab_transform[SRC_LANGUAGE].set_default_index(UNK_IDX)
vocab_transform[TGT_LANGUAGE].set_default_index(UNK_IDX)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_DIM = 256
N_HEAD = 8
NUM_ENCODER_LAYERS = 8
NUM_DECODER_LAYERS = 8
DROPOUT = 0.05
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_loaded = TransformerModel(src_vocab_size=SRC_VOCAB_SIZE, tgt_vocab_size=TGT_VOCAB_SIZE, 
            emb_dim=EMB_DIM, nhead=N_HEAD, num_encoder_layers=NUM_ENCODER_LAYERS, 
            num_decoder_layers=NUM_DECODER_LAYERS, dropout=DROPOUT)
model = model_loaded.to(device)
model_loaded.load_state_dict(torch.load('q2a_transformer_model.pth'))

<All keys matched successfully>

In [20]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        tgt_mask = (generate_square_subsequent_mask(ys.size(1))
                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.linear(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word[-1].item()
        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = token_transform[SRC_LANGUAGE](src_sentence)
    src = torch.tensor([[vocab_transform[SRC_LANGUAGE][token] for token in src]], dtype=torch.long)
    num_tokens = src.shape[1]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [22]:
sentence = 'Obama empfängt Netanyahu'
actual = 'Obama receives Netanyahu'

translate(model, sentence)

' of the cost of the cost of'

In [23]:
import pandas as pd

df_path = 'input.csv'
out_path = 'output.csv'
df = pd.read_csv(df_path, sep=',')

translated = []
for i in range(len(df)):
    translated.append(translate(model_loaded, df['de'][i]))

df['en'] = pd.Series(translated)
df.to_csv(out_path, index=False)

# Setup-2B

In [15]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")
model = model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
prefix = "translate English to German: "
sentence = "The house is wonderful."
input_ids = tokenizer(prefix + sentence, return_tensors="pt").input_ids
outputs = model.generate(input_ids.to(device))
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Das Haus ist wunderbar.


In [14]:
import pandas as pd

df_path = 'input.csv'
out_path = 'output.csv'
df = pd.read_csv(df_path, sep=',')

translated = []
for i in range(len(df)):
    prefix = "translate English to German: "
    sentence = df['de'][i]
    input_ids = tokenizer(prefix + sentence, return_tensors="pt").input_ids
    outputs = model.generate(input_ids.to(device))
    english_translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    translated.append(english_translation)

df['en'] = pd.Series(translated)
df.to_csv(out_path, index=False)

# Setup-2C

In [24]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5Config, AutoTokenizer
from torch.utils.data import Dataset, DataLoader

# Load the T5 tokenizer and model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model_loaded = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")
model_loaded = model_loaded.to(device)
model_loaded.load_state_dict(torch.load('q2c_t5_model.pth'))

<All keys matched successfully>

In [25]:
actual = 'Obama receives Netanyahu' 
sentence = 'Obama empfängt Netanyahu'

input_ids = tokenizer(sentence, return_tensors="pt").input_ids
input_ids = input_ids.to(device)
outputs = model_loaded.generate(input_ids)
print('=> Translated to English:')
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

=> Translated to English:
Obama senounces Netanyahu


In [11]:
import pandas as pd

df_path = 'input.csv'
out_path = 'output.csv'
df = pd.read_csv(df_path, sep=',')

translated = []
for i in range(len(df)):
    sentence = df['de'][i]
    input_ids = tokenizer(sentence, return_tensors="pt").input_ids
    outputs = model_loaded.generate(input_ids.to(device))
    english_translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    translated.append(english_translation)

df['en'] = pd.Series(translated)
df.to_csv(out_path, index=False)